# Chapter 4. Theoretical Distributions

## Imports

In [1]:
import numpy as np
from scipy.stats import norm, binom
from typing import Dict, List, Tuple

# Warning regarding solutions

NO GUARANTEE THAT THE SOLUTIONS WILL WORK OR WORK CORRECTLY! USE THEM AT YOUR OWN RISK!

THE ANSWERS PROVIDED BELOW MAY BE WRONG. USE THEM AT YOUR OWN RISK!

## Exercises

### Exercise 4.1

Assuming that the height of adult males has a Normal distribution,
what proportion of males will be more than two standard deviations
above the mean height?

#### Ex 4.1. Solution with scipy

In [ ]:
1 - norm.cdf(x=2, loc=0, scale=1)

### Answer to Exercise 4.1

Approximatelly 0.02275013194817921 or approx. 2.3% of men.

### Exercise 4.2

The probability of being blood group B is 0.08. What is the
probability that if one pint of blood is taken from each of
100 unrelated blood donors fewer than three pints of group B blood will be obtained?

#### Ex. 4.2. Solution with scipy

In [ ]:
binom.cdf(k=2, n=100, p=0.08)

#### Ex. 4.2. Solution by running n simulations

In [ ]:
def get_n_blood_pints(n: int = 100) -> np.ndarray:
    # 0 - blood group B, 1 - other blood group
    return np.random.choice(
        a=np.repeat(a=[0, 1], repeats=[8, 92]),
        size=n, replace=True)

In [ ]:
def get_counts(vector: np.ndarray) -> Dict[int, int]:
    uniques, counts = np.unique(vector, return_counts=True)
    return {k: v for (k, v) in zip(uniques, counts)}

In [ ]:
def is_gr_b_found(less_than: int = 3) -> bool:
    return get_counts(get_n_blood_pints()).get(0, 0) < less_than

In [ ]:
def get_n_simulations_for_gr_b(n_simuls: int = 1000) -> List[bool]:
    return [is_gr_b_found() for _ in range(n_simuls)]

In [ ]:
def get_probability_of_gr_b(n_simuls: int = 10_000) -> float:
    return np.mean(np.array(get_n_simulations_for_gr_b(n_simuls)))

In [ ]:
get_probability_of_gr_b()

### Answer to Exercise 4.2

The probability is around 0.01 or 1%

### Exercise 4.3

The probability of a baby being a boy is 0.52.
For six women delivering consecutively in the same labour ward on one day,
which of the following exact sequences of boys and girls is most likely and which least likely?

GBGBGB
BBBGGG
GBBBBB

In [ ]:
births_seqs: np.ndarray = np.array(["GBGBGB", "BBBGGG", "GBBBBB"])

#### Ex 4.3 Solution with mathematical calculations

In [ ]:
def get_prob_of_birth_seq(birth_seq: str) -> np.float64:
    return np.prod([0.52 if b == "B" else 0.48 for b in birth_seq])

In [ ]:
births_seqs_probs1: List[Tuple[float, str]] = [(get_prob_of_birth_seq(bs), bs) for bs in births_seqs]

#### Ex. 4.3 Solution with computer simulation

In [ ]:
def get_births_seq(how_many: int = 6) -> str:
    births: np.ndarray = np.random.choice(
        a = ["G", "B"], size=how_many, replace=True, p = [0.48, 0.52])
    return "".join(births)

In [ ]:
def get_n_birth_seqs(n: int = 100_000) -> np.ndarray:
    return np.array([get_births_seq() for _ in range(n)])

In [ ]:
rand_birth_seqs: np.ndarray = get_n_birth_seqs()
rand_births_counts: Dict[str, int] = {k: v for (k, v) in zip(*np.unique(rand_birth_seqs, return_counts=True))}
rand_births_probs: Dict[str, float] = {k: rand_births_counts.get(k, 0) / rand_birth_seqs.size for k in rand_births_counts}

In [ ]:
births_seqs_probs2: List[Tuple[float, str]] = [(rand_births_probs.get(bs, 0), bs) for bs in births_seqs]

#### Answer to Ex. 4.3

Assumming the exact order of births and the exact sex of the children (where p for B: 0.52, and for G: 0.48) we got:

According to mathematical calculations the probabilities are:
- (0.015550119935999997, 'GBGBGB')
- (0.015550119935999997, 'BBBGGG')
- (0.018249793536, 'GBBBBB')

 According to simulation [100'000 random birth sequences generated] the probabilities are:
- (0.01573, 'GBGBGB')
- (0.01541, 'BBBGGG')
- (0.01852, 'GBBBBB')

### Exercise 4.4

The Binomial distribution with p = 0.15 and n = 10

(a) 

If 15% of all pregnancies result in miscarriages,
what is the probability that more than half of a group of ten pregnant women will have a miscarriage?

(b)

Among groups of users of video display terminals there are 20'000 large enough for ten women to become pregnant in one year. If we call six or more miscarriages out of 10 a 'cluster', how many clusters would we expect in one year, assuming that there is no increased risc of miscarriage associated with using a terminal? (Based on Blackwell and Chang, 1988)